In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import src.gb_lag as lag

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
data = lag.feature_matrix(daily_threshold=500)

Formatting sales data...
Aggregating for monthly sales...


In [3]:
train_df, test_df = lag.stack_lag_dfs(data)

In [4]:
test_df.head()

year_month,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10
shop_item,,,,,,,,,,
0_1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0_10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0_1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0_10012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0_1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X = train_df.iloc[:, :-1].copy()
y = train_df.iloc[:, -1:].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=123)

In [6]:
# scale the data
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = MinMaxScaler()
y_train = sc_y.fit_transform(y_train)

In [7]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [22]:
# build LSTM
model = Sequential()
model.add(LSTM(units=1, 
               batch_input_shape=(1, X_train.shape[1], X_train.shape[2]),
#                return_sequences=True,
#                return_state=True,
               dropout=0.1,
               stateful=True))
model.add(Dense(1))
model.add(Dense(1))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [23]:
model.fit(X_train, y_train, 
          epochs=30, 
          batch_size=1, 
          verbose=1, 
          shuffle=False)

Epoch 1/30
678596/678596 [==============================] - 674s 994us/step - loss: 1.3681e-05
Epoch 2/30
678596/678596 [==============================] - 674s 993us/step - loss: 1.3618e-05s - l - ETA: 1s - loss: 1
Epoch 3/30
678596/678596 [==============================] - 674s 993us/step - loss: 1.3617e-05
Epoch 4/30
678596/678596 [==============================] - 674s 993us/step - loss: 1.3619e-05
Epoch 5/30
678596/678596 [==============================] - 674s 994us/step - loss: 1.3618e-05
Epoch 6/30
678596/678596 [==============================] - 675s 995us/step - loss: 1.3617e-05
Epoch 7/30
678596/678596 [==============================] - 675s 994us/step - loss: 1.3618e-05
Epoch 8/30
678596/678596 [==============================] - 762s 1ms/step - loss: 1.3614e-05
Epoch 9/30
678596/678596 [==============================] - 18841s 28ms/step - loss: 1.3617e-05
Epoch 10/30
678596/678596 [==============================] - 672s 990us/step - loss: 1.3620e-05
Epoch 11/30
678596/678596

In [24]:
y_pred = model.predict(X_test, batch_size=1)

In [25]:
y_pred_unscaled = sc_y.inverse_transform(y_pred)
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_unscaled))}')

RMSE: 3.5599907876187924


# Train on Entire Set

In [26]:
# scale
sc_X = MinMaxScaler()
X = sc_X.fit_transform(X)
# X_test = sc_X.transform(X_test)
sc_y = MinMaxScaler()
y = sc_y.fit_transform(y)

X_r = np.reshape(X, (X.shape[0], 1, X.shape[1]))

In [27]:
model.fit(X_r, y, 
          epochs=30, 
          batch_size=1, 
          verbose=1, 
          shuffle=False)

Epoch 1/30
848246/848246 [==============================] - 880s 1ms/step - loss: 1.0395e-05
Epoch 2/30
848246/848246 [==============================] - 854s 1ms/step - loss: 1.0728e-05
Epoch 3/30
848246/848246 [==============================] - 856s 1ms/step - loss: 1.0144e-05
Epoch 4/30
848246/848246 [==============================] - 853s 1ms/step - loss: 1.0243e-05
Epoch 5/30
848246/848246 [==============================] - 856s 1ms/step - loss: 1.0378e-05
Epoch 6/30
848246/848246 [==============================] - 2536s 3ms/step - loss: 9.7719e-06
Epoch 7/30
848246/848246 [==============================] - 849s 1ms/step - loss: 1.0231e-05
Epoch 8/30
848246/848246 [==============================] - 871s 1ms/step - loss: 9.8764e-06
Epoch 9/30
848246/848246 [==============================] - 851s 1ms/step - loss: 1.0383e-05
Epoch 10/30
848246/848246 [==============================] - 846s 997us/step - loss: 1.0185e-05
Epoch 11/30
848246/848246 [==============================] - 846s 

# Test Set
Merge test set to 2015 data. Then get predictions.

In [28]:
test_set = pd.read_csv('data/test.csv')

test_set['shop_item'] = \
    test_set['shop_id'].astype(str) + '_' + test_set['item_id'].astype(str)

test_df = test_df.reset_index()

In [29]:
test_df.head()

year_month,shop_item,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10
0,0_1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0_10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0_1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0_10012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0_1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
test_input = pd.merge(test_set.loc[:, 'shop_item'], 
                    test_df, 
                    on='shop_item', how='left')

test_input.fillna(0, inplace=True)

In [31]:
test_input.head()

,shop_item,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10
0,5_5037,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,5_5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5_5233,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,5_5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5_5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
X_test = test_input.iloc[:, 1:]
X_test = sc_X.transform(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [33]:
y_pred = model.predict(X_test, batch_size=1)
y_pred = sc_y.inverse_transform(y_pred)

In [34]:
test_set['item_cnt_month'] = y_pred
test_set.head()

,ID,shop_id,item_id,shop_item,item_cnt_month
0,0,5,5037,5_5037,-0.020541
1,1,5,5320,5_5320,-0.019969
2,2,5,5233,5_5233,-0.021411
3,3,5,5232,5_5232,-0.019818
4,4,5,5268,5_5268,-0.019748


In [35]:
output = test_set.loc[:, ['ID', 'item_cnt_month']].copy()
output.set_index(keys='ID', inplace=True)
output = output.clip(0, 20)

output.to_csv('output/lstm_all_data_v2.csv', index=True)